In [2]:
import pandas as pd
import censusdata

In [31]:
tables = censusdata.search('acs5', 2019,'concept', 'divorce')
tables

[('B12503_001E',
  'DIVORCES IN THE LAST YEAR BY SEX BY MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER',
  'Estimate!!Total:'),
 ('B12503_002E',
  'DIVORCES IN THE LAST YEAR BY SEX BY MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER',
  'Estimate!!Total:!!Male:'),
 ('B12503_003E',
  'DIVORCES IN THE LAST YEAR BY SEX BY MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER',
  'Estimate!!Total:!!Male:!!Never married'),
 ('B12503_004E',
  'DIVORCES IN THE LAST YEAR BY SEX BY MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER',
  'Estimate!!Total:!!Male:!!Ever married:'),
 ('B12503_005E',
  'DIVORCES IN THE LAST YEAR BY SEX BY MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER',
  'Estimate!!Total:!!Male:!!Ever married:!!Divorced last year'),
 ('B12503_006E',
  'DIVORCES IN THE LAST YEAR BY SEX BY MARITAL STATUS FOR THE POPULATION 15 YEARS AND OVER',
  'Estimate!!Total:!!Male:!!Ever married:!!Not divorced last year'),
 ('B12503_007E',
  'DIVORCES IN THE LAST YEAR BY SEX BY MARITAL

In [27]:
def get_code(name, race):
    tables = censusdata.search('acs5', 2019,'concept', name)
    full_name = name + " (" + race + ")"
    codes = []
    for item in tables:
        if item[1] == full_name:
            codes.append(item[0])
    return codes

In [28]:
name = 'MEDIAN AGE AT FIRST MARRIAGE'
r = 'AMERICAN INDIAN AND ALASKA NATIVE ALONE'
get_code(name, r)

['B12007C_001E', 'B12007C_002E']

In [ ]:
table_vars = {"MEDIAN AGE AT FIRST MARRIAGE": ['Male age', 'Female age'],
              
             
             }

Then we can download the variables we choose into a pandas data frame by using the codes above.

In [7]:
"""
This function downloads the specified tables from the specified years and reformats them

Inputs: codes - a list of the table codes
        names - what to rename the variables for each table code
        years - years to get a table from
        
Output: one dataframe with the requested data compiling the different years
"""
def get_tables(codes, names, years):
    tables = []
    for year in years:
        #Get table
        df = censusdata.download('acs5', year,
                   censusdata.censusgeo([('state', '*')]),
                    codes)
        
        #Rename columns
        name_dict = dict(zip(codes, names))
        name_dict['index'] = 'State'
        df = df.reset_index() #Turns row names into row
        df = df.rename(columns = name_dict)
        
        #Shorten states column to state name
        df = df.astype({'State':'str'})
        df['State'] = df['State'].str.split(':').str.get(0) 
        
        #Add column for year
        df['Year'] = year
        
        tables.append(df)
    return pd.concat(tables)

In [6]:
marriage_2019 = censusdata.download('acs5', 2019,
                   censusdata.censusgeo([('state', '*')]),
                    ['B12007_001E', 'B12007_002E'])
marriage_2019.head()

,B12007_001E,B12007_002E
"Alabama: Summary level: 040, state:01",28.5,26.7
"Alaska: Summary level: 040, state:02",29.2,26.4
"Arizona: Summary level: 040, state:04",29.9,27.8
"Arkansas: Summary level: 040, state:05",27.2,25.7
"California: Summary level: 040, state:06",30.8,29.0


## Streamlining this process

To reformat this table and others, we wrote the following function. We can follow the same process to find other tables and variables of interest, and then plug that information into this function to get a nicer table.

In [8]:
marriage = get_tables(['B12007_001E', 'B12007_002E'], ['Male age', 'Female age'], [2009, 2014, 2019])
marriage.head()

,State,Male age,Female age,Year
0,Alaska,27.2,25.2,2009
1,Alabama,26.8,25.3,2009
2,Arkansas,25.8,24.3,2009
3,Arizona,27.8,25.8,2009
4,California,28.8,26.8,2009


In [11]:
household_type = get_tables(['B11001_001E', 'B11001_002E','B11001_003E','B11001_004E','B11001_007E','B11001_008E','B11001_009E'], ['Total', 'Total Family','Married-couple Family', 'Single Householder, no spouse','Total Nonfamily','Nonfamily Living Alone','Nonfamily Not Alone'], [2009, 2014, 2019])
household_type

,State,Total,Total Family,Married-couple Family,"Single Householder, no spouse",Total Nonfamily,Nonfamily Living Alone,Nonfamily Not Alone,Year
0,Alaska,234779,159319,118716,40603,75460,57718,17742,2009
1,Alabama,1819441,1236035,894351,341684,583406,508317,75089,2009
2,Arkansas,1109635,754486,563199,191287,355149,305252,49897,2009
3,Arizona,2248170,1492544,1115833,376711,755626,603300,152326,2009
4,California,12187191,8333690,6085094,2248596,3853501,2993951,859550,2009
...,...,...,...,...,...,...,...,...,...
47,Washington,2848396,1841954,1430460,411494,1006442,759370,247072,2019
48,West Virginia,732585,473856,356024,117832,258729,217699,41030,2019
49,Wisconsin,2358156,1482213,1148844,333369,875943,696118,179825,2019
50,Wyoming,230101,148652,119353,29299,81449,64997,16452,2019


In [12]:
divorces = get_tables(['B12503_001E','B12503_003E','B12503_005E','B12503_006E','B12503_008E','B12503_010E','B12503_011E'],
                      ['Total','Male Never Married', 'Male Married; Divorced Last Year', 'Male Married; Not Divorced Last Year','Female Never Married','Female Married; Divorced Last Year','Female Married; Not Divorced Last Year'],
                      [2012, 2019])
divorces

,State,Total,Male Never Married,Male Married; Divorced Last Year,Male Married; Not Divorced Last Year,Female Never Married,Female Married; Divorced Last Year,Female Married; Not Divorced Last Year,Year
0,Alabama,3844391,584355,22564,1234437,517693,26151,1459191,2012
1,Alaska,556204,105333,3389,181330,74461,3330,188361,2012
2,Arizona,5056561,876661,24077,1595535,711047,27608,1821633,2012
3,Arkansas,2325562,330745,13975,784152,276122,16462,904106,2012
4,California,29700084,5778554,112136,8772035,4819717,129478,10088164,2012
...,...,...,...,...,...,...,...,...,...
47,Washington,6031108,1042361,23783,1934478,826429,26890,2177167,2019
48,West Virginia,1512469,232652,7665,502692,183487,7214,578759,2019
49,Wisconsin,4734360,825525,15745,1497749,695245,15502,1684594,2019
50,Wyoming,466549,72140,2069,163131,53486,2687,173036,2019


## Data visualization

Now we can make charts from the data.

In [ ]:
from matplotlib import pyplot as plt
import plotly.io as pio
from plotly import express as px

### 1. Median Age of Marriage in California

In [ ]:
mar_cal = marriage[marriage['State'] == "California"]
mar_cal

In [ ]:
fig = px.scatter(data_frame = mar_cal, 
                x = "Year", 
                y = ["Male age", "Female age"],
                title = "Median Age of First Marriage (CA)",
                trendline = "ols", # ordinary least squares regression trendline
                width = 800,
                height = 600)

fig.show()

### 2. Frequency of Different Household Types in California

In [ ]:
household_cal = household_type[household_type['State']=="California"]
household_cal

In [ ]:
mc_percentage = household_cal['Married-couple Family'] / household_cal['Total']
sh_percentage = household_cal['Single Householder, no spouse'] / household_cal['Total']
nla_percentage = household_cal['Nonfamily Living Alone'] / household_cal['Total']
nna_percentage = household_cal['Nonfamily Not Alone'] / household_cal['Total']
year = household_cal['Year']

household_percentage = pd.DataFrame({
    'Married-couple Family': mc_percentage,
    'Single Householder': sh_percentage, 
    'Nonfamily Living Alone': nla_percentage, 
    'Nonfamily Not Alone': nna_percentage,
    'Year': year
})

household_percentage

In [ ]:
household_percentage = household_percentage.round(decimals = 4)

In [ ]:
fig = px.bar(household_percentage,
             x="Year", 
             y=["Married-couple Family","Single Householder", "Nonfamily Living Alone","Nonfamily Not Alone"],  
             title="Household Types (CA)")
fig.show()

### 3. Divorces in the Last Year in California

In [ ]:
divorces_cal = divorces[divorces['State']=='California']
divorces_cal

In [ ]:
never_married_total = divorces_cal["Male Never Married"] + divorces_cal["Female Never Married"]
div_last_year = divorces_cal["Male Married; Divorced Last Year"] + divorces_cal["Female Married; Divorced Last Year"]
married_not_div = divorces_cal["Male Married; Not Divorced Last Year"] + divorces_cal["Female Married; Not Divorced Last Year"]
total = divorces_cal["Total"]
year = divorces_cal["Year"]

divorces_cal_totals = pd.DataFrame({
    'Total': total,
    'Never Married Total': never_married_total,
    'Ever Married; Divorced Last Year': div_last_year, 
    'Ever Married; Did Not Divorce Last Year': married_not_div, 
    'Year': year
})


In [ ]:
never_married_per = divorces_cal_totals["Never Married Total"] / divorces_cal_totals["Total"]
married_div_per = divorces_cal_totals["Ever Married; Divorced Last Year"] / divorces_cal_totals["Total"]
married_not_div_per = divorces_cal_totals["Ever Married; Did Not Divorce Last Year"] / divorces_cal_totals["Total"]

divorces_cal_percentages = pd.DataFrame({
    'Never Married': never_married_per,
    'Ever Married; Divorced Last Year': married_div_per, 
    'Ever Married; Did Not Divorce Last Year': married_not_div_per, 
    'Year': year
})

divorces_cal_percentages
divorces_cal_percentages = divorces_cal_percentages.round(decimals = 4)
divorces_cal_percentages

In [ ]:
fig = px.scatter(divorces_cal_percentages, 
                x = "Year", 
                y = ["Never Married", "Ever Married; Divorced Last Year", "Ever Married; Did Not Divorce Last Year"],
                title = "Divorces in the Past Year in CA",
                trendline = "ols", # ordinary least squares regression trendline
                width = 800,
                height = 600)

fig.show()